In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
import shap
import matplotlib.pyplot as plt
import statsmodels.api as sm
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge, Lasso, QuantileRegressor, LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from linearmodels.panel import PanelOLS
from statsmodels.stats.outliers_influence import variance_inflation_factor
from numpy.linalg import matrix_rank

/Users/apple/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Read Data

path='./Data/merged_burden_risk.csv'

df = pd.read_csv(path)

In [ ]:
#rename columns for ease 

df = df.dropna(subset=['dalys_(disability-adjusted_life_years)'])
df.rename(columns={'Chronic_Respiratory(RSD)': 'RSD'}, inplace=True)
df.rename(columns={'dalys_(disability-adjusted_life_years)': 'DALYs'}, inplace=True)
df = df.sort_values(by=['country', 'year'])

In [ ]:
#filling in missing data for HAQ_Index and RSD using Linear Interpolation and Regression Imputation

# Step 1: Linear interpolation (within each country)
for col in ['HAQ_Index', 'RSD']:
    df[col] = df.groupby('country')[col].transform(lambda x: x.interpolate(method='linear', limit_direction='both'))

# Step 2: Fill remaining HAQ using regression on year
if df['HAQ_Index'].isna().sum() > 0:
    known = df[df['HAQ_Index'].notna()]
    unknown = df[df['HAQ_Index'].isna()]
    model = LinearRegression().fit(known[['year']], known['HAQ_Index'])
    df.loc[unknown.index, 'HAQ_Index'] = model.predict(unknown[['year']])

# Step 3: Fill remaining RSD (if any) using regression
if df['RSD'].isna().sum() > 0:
    known_rsd = df[df['RSD'].notna()]
    unknown_rsd = df[df['RSD'].isna()]
    model_rsd = LinearRegression().fit(known_rsd[['year']], known_rsd['RSD'])
    df.loc[unknown_rsd.index, 'RSD'] = model_rsd.predict(unknown_rsd[['year']])

In [ ]:
#Log Transformations: Normalize skewed variables
df['log_gdp_per_capita'] = np.log(df['GDP PER CAPITA (USD)'] + 1)
df['log_population_density'] = np.log(df['Population Density'] + 1)
df['log_total_co2'] = np.log(df['Total CO2 Emission excluding LUCF (Mt)'] + 1)


#Per Capita Pollution Measures: Scale pollution to population for fair comparisons
df['co2_per_capita'] = df['Total CO2 Emission excluding LUCF (Mt)'] / df['Population']
df['no2_per_capita'] = df['Nitrogen oxide'] / df['Population']
df['black_carbon_per_capita'] = df['Black Carbon'] / df['Population']

#Pollution burden adjusted for healthcare quality
df['pollution_x_low_haq'] = df['co2_per_capita'] * (1 - df['HAQ_Index'] / 100)

df['year_index'] = df['year'] - df['year'].min() #Time Index: Relative year index

df['lagged_dalys'] = df.groupby('country')['DALYs'].shift(1) #Lagged DALYs: Previous year's burden for temporal modeling

df.head()


In [ ]:
#Rolling Averages: Capture long-term exposure effects
df['pm25_3yr_avg'] = df.groupby('country')['pm25_DALY'].transform(lambda x: x.rolling(window=3, min_periods=1).mean())
df['dalys_3yr_avg'] = df.groupby('country')['DALYs'].transform(lambda x: x.rolling(window=3, min_periods=1).mean())

#Temporal Change: Year-over-year change in pollution
df['delta_pm25'] = df.groupby('country')['pm25_DALY'].diff()
df['delta_black_carbon'] = df.groupby('country')['Black Carbon'].diff()

#Interaction Terms: Capture compound effects between variables
df['gdp_x_haq'] = df['GDP PER CAPITA (USD)'] * df['HAQ_Index']
df['smoking_x_pm25'] = df['smoking_DALY'] * df['pm25_DALY']
df['haq_x_dalys_lag'] = df['HAQ_Index'] * df.groupby('country')['DALYs'].shift(1)

#Vulnerability Index: Composite of low GDP, high population density, and low HAQ
df['norm_gdp'] = df.groupby('year')['GDP PER CAPITA (USD)'].transform(lambda x: (x - x.min()) / (x.max() - x.min()))
df['norm_density'] = df.groupby('year')['Population Density'].transform(lambda x: (x - x.min()) / (x.max() - x.min()))
df['norm_haq'] = df.groupby('year')['HAQ_Index'].transform(lambda x: (x - x.min()) / (x.max() - x.min()))
df['vulnerability_index'] = (1 - df['norm_gdp']) + df['norm_density'] + (1 - df['norm_haq'])


# Preview key new features
df[['log_gdp_per_capita', 'co2_per_capita', 'pollution_x_low_haq', 'pm25_3yr_avg',
    'delta_pm25', 'gdp_x_haq', 'vulnerability_index']].head()

In [ ]:
all_features = [
    'log_gdp_per_capita', 'log_population_density', 'log_total_co2',
    'co2_per_capita', 'pollution_x_low_haq', 'year_index', 'lagged_dalys',
    'pm25_3yr_avg', 'delta_pm25', 'gdp_x_haq', 'smoking_x_pm25',
    'haq_x_dalys_lag', 'vulnerability_index'
]

structural_features = [f for f in all_features if f not in ['lagged_dalys', 'haq_x_dalys_lag']]

In [ ]:
#Panel Modeling — Random Effects Approximation (OLS + Clustered SE)
# STEP 6A: Random Effects-style Panel Regression (with Clustered SEs)


# Feature list
candidate_features = [
    'log_gdp_per_capita',
    'pollution_x_low_haq',
    'pm25_3yr_avg',
    'smoking_x_pm25',
    'vulnerability_index'
]

# Prepare panel structure
panel_df = df[candidate_features + ['DALYs']].dropna().copy()
panel_df['const'] = 1

# Remove collinear feature
final_features = candidate_features.copy()
for feature in candidate_features:
    test_features = [f for f in final_features if f != feature]
    X = panel_df[test_features + ['const']]
    if matrix_rank(X.values) == len(test_features) + 1:
        final_features = test_features
        break

# Final regression design
X_final = panel_df[final_features + ['const']]

# VIF check
vif_data = pd.DataFrame()
vif_data["feature"] = final_features
vif_data["VIF"] = [variance_inflation_factor(X_final[final_features].values, i) for i in range(len(final_features))]

# Run PanelOLS
model = PanelOLS(panel_df['DALYs'], X_final, entity_effects=True)
results = model.fit(cov_type='clustered', cluster_entity=True)

# Output
print(results.summary)
print(vif_data)
